In [9]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [10]:
start_page = 1 
end_page = 2

In [21]:
def get_crypto_url(start_page, end_page):
    base_url = 'https://coinmarketcap.com/'
    total_storage = []
    for i in range(start_page,end_page):
        response = requests.get(base_url + str(i)+"/")
        soup = BeautifulSoup(response.text)
        link_list = soup.find_all('a',{'class':'cmc-link'})
        cryptocurrency_link = []
        for link in link_list:
            try:
                cryptocurrency_link.append(link['href'])
            except:
                continue
        filter_crypto_link = [link for link in cryptocurrency_link if '/currencies' in link 
                              and 'markets' not in link and 'volume' not in link
                             and 'social' not in link
                             and 'news' not in link
                             and 'historical' not in link
                             and 'ratings' not in link
                             and 'index' not in link
                             and 'analysis' not in link]
        filter_crypto_link = list(set(filter_crypto_link))
        total_storage.extend(filter_crypto_link)
        
    return total_storage

def get_coin_info(soup_text):
    text = soup_text
    storage_dict = {}
    rank = re.compile('CoinMarketCap ranking is #(\d+)')
    circulating_supply =  re.compile('circulating supply of ([\d,]+)')
    total_supply = re.compile('max. supply of ([\d,]+)')
    market_cap = re.compile('market cap of \$([\d,]+)')
    #top_exchagnes = re.compile('top exchanges for trading in [\w\s]+ are currently ([\w\s,]+.?[\w\s,]+).')
    #details = re.compile('What is [\w\s()]+\?(.+) How do you buy') not general enough
    top_exchagnes = re.compile('top exchanges for trading in [\w\s]+ are currently (.+)You can find')
    details = re.compile('(What [I|i]s .+)')
    pattern = {
        'CoinMarketCap ranking':rank,
           'Circulating supply':circulating_supply,
           'Total supply':total_supply,
           'market cap USD':market_cap,
           'top_exchagnes':top_exchagnes,
           'details':details
               
          }
    for key,each_pattern in pattern.items():
        try:
            storage_dict[key] = each_pattern.findall(text)[0]
        
        except:
            storage_dict[key] = ''
    return storage_dict  

In [22]:
link_list = get_crypto_url(1, 2)

In [23]:
base_url = 'https://coinmarketcap.com'
storage = []
for link in link_list:
    url = base_url + link
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    soup_text = soup.get_text()
    coin_info = get_coin_info(soup_text)
    #add name
    name = soup.find('h2').get_text()
    ticker_pattern = re.compile('[A-Z]{2,}')
    ticker = ticker_pattern.findall(name)[0]
    name = name[0:-len(ticker)]
    coin_info['name'] = name
    coin_info['ticker'] = ticker
    coin_info['link'] = url
    storage.append(coin_info)
data = pd.DataFrame(storage)

In [24]:
data

,CoinMarketCap ranking,Circulating supply,Total supply,market cap USD,top_exchagnes,details,name,ticker,link
0,89,"11,000,000",,"198,134,951","Binance, Binance.KR, OKEx, CoinBene, and CoinT...",,Augur,REP,https://coinmarketcap.com/currencies/augur/
1,85,"30,062,138","100,000,000","209,078,413","Uniswap (V2), KuCoin, Liquid, Hotbit, and BitM...",,Energy Web Token,EWT,https://coinmarketcap.com/currencies/energy-we...
2,72,"749,772,094","1,000,000,000","322,501,520","Binance, Huobi Global, Binance.KR, OKEx, and B...",,0x,ZRX,https://coinmarketcap.com/currencies/0x/
3,86,"20,494,033",,"203,595,153","Binance, Huobi Global, OKEx, BitZ, and HBTC.",,Band Protocol,BAND,https://coinmarketcap.com/currencies/band-prot...
4,62,"1,245,666,867","10,000,000,000","408,995,974","Binance, Huobi Global, OKEx, CoinBene, and FTX.",,The Graph,GRT,https://coinmarketcap.com/currencies/the-graph/
...,...,...,...,...,...,...,...,...,...
95,95,"1,494,082,151",,"177,172,244","Binance, Huobi Global, OKEx, BitZ, and HBTC.",,Decentraland,MANA,https://coinmarketcap.com/currencies/decentral...
96,31,"208,501,479",,"1,220,114,851","Binance, Huobi Global, Binance.KR, OKEx, and B...",,Cosmos,ATOM,https://coinmarketcap.com/currencies/cosmos/
97,46,"4,428,169","10,000,000","727,735,712","Binance, Huobi Global, Binance.KR, OKEx, and B...",,Compound,COMP,https://coinmarketcap.com/currencies/compound/
98,90,"184,814,951",,"186,264,730","Uniswap (V2), and Bittrex.",,Terra,USDUST,https://coinmarketcap.com/currencies/terrausd/


In [20]:
data.to_excel('CoinMarketCap_1.xlsx',index = False)